In [0]:
import pandas as pd
import numpy as np
from sklearn import datasets
from numpy.linalg import inv
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [0]:
def hypothesis(X, coeff):
  return np.dot(X, coeff)

In [0]:
# Feature scaling
def mean(a):
  mean = float(np.mean(a))
  func = np.vectorize(lambda t: t - mean)
  return func(a)

def std_var(a):
  std = float(np.std(a))
  func = np.vectorize(lambda t: t / std)
  return func(a)

def feature_scaling(X):
  X = np.apply_along_axis(mean, 0, X)
  X = np.apply_along_axis(std_var, 0, X)
  return X

In [0]:
def normalizeXy(X, y):
#   Change as per notation to get: w0x0 + w1x1 + w2x2 + ...
# i.e. add an extra feature vector x0 for bias
# Also do feature scaling
#   assert X.shape == (303, 13)
  
  shape = (X.shape[0], 1)
#   assert shape == (303, 1)
  
  newX = feature_scaling(X)
  newX = np.hstack((np.ones(shape), newX))
  
#   assert newX.shape == (303, 14)
  
  newy = y.reshape((-1,1))
#   assert newy.shape == (303, 1)
  
  return newX, newy
  

In [0]:
# Method 1 to find theta: Direct Normal Equation

def findTheta(X, y):
#   Instead of using gradient descent to estimate theta, use normal equation to get theta
# theta = ((Xt.X)^-1).Xt.y
  pinv = inv(np.dot(X.T, X))
  nextDot = np.dot(pinv, X.T)
  return np.dot(nextDot, y)

In [0]:
# Method 2 to find theta: Gradient descent

# loss: (303, 1)
# X: (303, 14)

def hypothesis(X, coeff):
  return np.dot(X, coeff)

def gradient(X, y, theta):
  m = float(X.shape[0])
  hx = hypothesis(X, theta) 
  loss = hx - y
  grad =  1/m * np.dot(X.T, loss)
  return grad 

def cost_function(X, y, theta): 
  m = float(X.shape[0])
  hx = hypothesis(X, theta) 
  loss = hx - y
  cost = (1/2*m) * np.sum(np.square(loss))
  return cost

def gradient_descent(X, y, coeff, learning_rate = 0.01, min_cost_change = 0.0001, max_epochs = 50000):
  epoch = 1 
  current_cost = cost_function(X, y, coeff) 
  cost_change = learning_rate
  
#   for epoch in range(0,1000): 
  while cost_change > min_cost_change and epoch < max_epochs:
    prev_cost = current_cost
    grad = gradient(X, y, coeff)
    coeff = coeff - (learning_rate * grad) 
    current_cost = cost_function(X, y, coeff) 
    cost_change = prev_cost - current_cost
#     print("cost_change:", cost_change)
    epoch += 1
  
  return coeff, epoch 

In [0]:
# # Method 2 to find theta: Mini batch gradient descent
# def calc_cost(X, y, theta): 
#   m = float(X.shape[0])
#   hx = hypothesis(X, theta) 
#   cost = (1/2*m) * np.sum(np.square(hx - y))
  
# def create_mini_batches(X, y, batch_size): 
#     mini_batches = []
#     data = np.hstack((X, y)) 
#     np.random.shuffle(data) 
#     n_minibatches = data.shape[0] // batch_size 
#     i = 0
  
#     for i in range(n_minibatches + 1): 
#       if data.shape[0] % batch_size != 0: 
#         mini_batch = data[i * batch_size:data.shape[0]]
#       else:
#         mini_batch = data[i * batch_size:(i + 1)*batch_size, :] 
      
#       X_mini = mini_batch[:, :-1] 
#       Y_mini = mini_batch[:, -1].reshape((-1, 1)) 
#       mini_batches.append((X_mini, Y_mini))
#     return mini_batches 
  
# def mini_batch_gradient_descent(X, y, learning_rate = 0.001, batch_size = 32): 
#     theta = np.zeros((X.shape[1], 1)) 
#     cost_history = [] 
#     max_iters = 3
#     for itr in range(max_iters): 
#         mini_batches = create_mini_batches(X, y, batch_size) 
#         for mini_batch in mini_batches: 
#             X_mini, y_mini = mini_batch 
#             theta = theta - learning_rate * gradient(X_mini, y_mini, theta) 
#             cost_history.append(calc_cost(X_mini, y_mini, theta)) 
  
#     return theta, cost_history 

In [0]:
def predict_y(X, coeff): 
    pred_value = hypothesis(X, coeff) 
    return pred_value

In [0]:
if __name__ == "__main__":
  dataset = datasets.load_boston()
  X = dataset.data
  y = dataset.target
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
  print("X_train.shape: ", X_train.shape)
  print("y_train.shape: ", y_train.shape)
  
  X_train, y_train = normalizeXy(X_train, y_train)
  print("New X_train.shape after normalization: ", X_train.shape)
  print("New y_train.shape after normalization: ", y_train.shape)
  
#   coeff = findTheta(X_train, y_train)

#   Initial values
  coeff = np.random.rand(X_train.shape[1], 1)
  coeff, epoch = gradient_descent(X_train, y_train, coeff)
  print("\ncoeff.shape: ", coeff.shape)
  print("coefficients: ", coeff)
  print("epoch: ",epoch)
  

  y_pred = predict_y(X_train, coeff)
#   assert(y_pred.shape == (303,1))
  
  print("\ny actual vs predicted:")
  print(y_train[:3])
  print(y_pred[:3])
  
  rmse = np.sqrt(metrics.mean_squared_error(y_train, y_pred))
  print("\nrmse: ", format(rmse, '.5g'))
  


X_train.shape:  (404, 13)
y_train.shape:  (404,)
New X_train.shape after normalization:  (404, 14)
New y_train.shape after normalization:  (404, 1)

coeff.shape:  (14, 1)
coefficients:  [[22.5730198 ]
 [-0.77618933]
 [ 0.81012723]
 [ 0.14594059]
 [ 0.25528977]
 [-1.88666879]
 [ 3.18390405]
 [-0.14780517]
 [-2.72987965]
 [ 2.20934025]
 [-2.00794581]
 [-2.3085894 ]
 [ 0.78881118]
 [-3.03760937]]
epoch:  9895

y actual vs predicted:
[[23. ]
 [19.6]
 [22. ]]
[[19.6613528 ]
 [18.33184451]
 [29.22369433]]

rmse:  4.5176


In [0]:
# Using scikit libraries:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

X2_train, X2_test, y2_train, y2_test = train_test_split(X, y, test_size=0.2)

scaler = StandardScaler()
X2_train = scaler.fit_transform(X2_train)

reg = LinearRegression().fit(X2_train, y2_train)
y2_pred = reg.predict(X2_train)

print("\ny actual vs predicted:")
print(y2_train[:3])
print(y2_pred[:3])
  
rmse = np.sqrt(metrics.mean_squared_error(y2_train, y2_pred))
print("\nrmse: ", format(rmse, '.5g'))


y actual vs predicted:
[16.1 13.1 20.3]
[19.13827971 14.42493049 19.71903769]

rmse:  4.7684
